In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import neurokit2 as nk
from datetime import datetime
import pandas as pd

In [5]:
data = pd.read_csv('C:/Users/eleon/Desktop/SDAP/backend/src/data/p1_wk1/1022102cFnorm.csv', usecols=['MR', 'ML', 'ECG'])
loc = pd.read_csv('C:/Users/eleon/Desktop/SDAP/backend/src/data/p1_wk1/1022102clocation_Bites.csv')

In [6]:
data_5_min = data[:600000]

In [7]:
data_5_min

,MR,ML,ECG
0,0.060487,-0.016969,0.234100
1,0.658350,-0.184600,0.234080
2,3.215500,-0.900960,0.234100
3,9.209700,-2.577700,0.234110
4,16.877000,-4.715300,0.234090
...,...,...,...
599995,0.000559,-0.013620,0.000705
599996,-0.000154,-0.002426,0.000699
599997,0.006617,0.005797,0.000696
599998,0.009843,-0.000941,0.000694


# Rectified signal

In [9]:
rectified_signals = np.abs(data_5_min)

# RMS

In [10]:
# Define a window size (in number of samples)
# 60-120 ms?
window_size = int(0.070*2000)

# Compute RMS with a rolling window
rms = rectified_signals.rolling(window=window_size, min_periods=1).apply(lambda x: np.sqrt(np.mean(x**2)), raw=True)

# Check the output size
print(f"Original signal length: {len(rectified_signals)}")
print(f"RMS signal length: {len(rms)}")

140
Original signal length: 600000
RMS signal length: 600000


In [11]:
rms

,MR,ML,ECG
0,0.060487,0.016969,0.234100
1,0.467484,0.131082,0.234090
2,1.895303,0.531066,0.234093
3,4.888637,1.368451,0.234098
4,8.722709,2.438224,0.234096
...,...,...,...
599995,0.028857,0.015691,0.000710
599996,0.028824,0.015638,0.000710
599997,0.028822,0.015505,0.000710
599998,0.028831,0.015255,0.000710


# FIND MVC (Maximum Volontary Contraction)

In [13]:
# Define Maximum Volontary Contraction
mvc_MR = 0
mvc_ML = 0
for index, row in loc.iterrows():
    begin_contraction, end_contraction = int(row[0]), int(row[1])


    bite_data = rms[begin_contraction:end_contraction]

    max_in_bite_MR = bite_data['MR'].max()
    max_in_bite_ML = bite_data['ML'].max()

    print(max_in_bite_MR)
    print(max_in_bite_ML)


    # TODO: check if average of ML and MR max row is okay
    # max_in_bite_avg = (max_in_bite['MR'][0] + max_in_bite['ML'][0]) / 2

    if float(max_in_bite_MR) > float(mvc_MR):
        mvc_MR = max_in_bite_MR

    if float(max_in_bite_ML) > float(mvc_ML):
        mvc_MR = max_in_bite_ML
    

0.9604318320957134
1.4121848163703676


C:\Users\eleon\AppData\Local\Temp\ipykernel_35720\3399387391.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  begin_contraction, end_contraction = int(row[0]), int(row[1])


TypeError: cannot convert the series to <class 'float'>